###  3 - Parsing of text to table

In [1]:
import pandas as pd
import json
import os
import io
from shutil import copyfile
import re
import itertools
from google.cloud import vision
from datetime import datetime
import numpy as np
from google.protobuf.json_format import MessageToDict
pd.set_option('display.max_rows', 10)

### Parsing algorithms for each mill
Input: Raw text in string format  
Output: list of date, entry and exit times and weights  
One algorithm per mill because of the variability of receipt layouts

In [17]:
def parsing_skip(text):
    res = []
    dates = re.findall('[0-9]{1,2}\/[0-9]{1,2}\/[0-9]{4}', text)
    try:
        date = dates[0]
        date = datetime.strptime(date, '%d/%m/%Y').strftime('%m/%d/%Y') # convert to American date format
    except:
        date = ''
    res.append(date)
    times = re.findall('[0-9]{1,2}\:[0-9]{1,2}\:[0-9]{1,2}|[0-9]{1,2}\.[0-9]{1,2}\:[0-9]{1,2}|[0-9]{1,2}\.[0-9]{1,2}\.[0-9]{1,2}|[0-9]{1,2}\:[0-9]{1,2}\.[0-9]{1,2}', text)
    try:
        entry_time = times[0].replace('.',':')
    except IndexError:
        entry_time =''
    try:
        exit_time = times[1].replace('.',':')
    except IndexError:
        exit_time = ''
    res.append(entry_time)
    res.append(exit_time)
    weights = re.findall('\d+\.\d+ kg|\d+ kg|\d+\.\d+ 1g|\d+ 1g|\d+\.\d+ ko|\d+ ko', text)
    weights = [int(w.replace('.','').replace(' kg','').replace(' 1g','').replace(' ko','')) for w in weights]
    while len(weights) < 5:
        weights.append('')
    res += weights
    return res

In [18]:
def parsing_nhr(text):
    res = []
    dates = re.findall('[0-9]{1,2}\/[0-9]{1,2}\/[0-9]{4}', text)
    try:
        date = dates[0]
        try:
            date = datetime.strptime(date, '%d/%m/%Y').strftime('%m/%d/%Y') # convert to American date format
        except ValueError:
            pass
    except IndexError:
        date = ''
    res.append(date)
    times = re.findall('\d+\:\d+\:\d+', text)
    try:
        entry_time = times[0]
    except IndexError:
        entry_time =''
    try:
        exit_time = times[1]
    except IndexError:
        exit_time = ''
    res.append(entry_time)
    res.append(exit_time)
    weights = re.findall('\d+\,[0-9]{3}|\d+\.\[0-9]{3}', text)
    try:
        first_weight = weights[0]
        text = text.replace(':','')
        text_list = text.split('\n')
        idx = text_list.index(first_weight)
        weights = text_list[idx:idx+5]
        weights = [w.replace('.','').replace(',','') for w in weights]
        weights = [w for w in weights if w.isdigit()]
        weights = [int(w) for w in weights]
    except IndexError:
        pass
    while len(weights) < 5:
        weights.append('')
    res += weights
    return res

In [19]:
def parsing_arvena(text):
    res = []
    dates = re.findall('[0-9]{1,2}\-[0-9]{1,2}\-[0-9]{4}', text)
    try:
        date = dates[0]
        date = datetime.strptime(date, '%d-%m-%Y').strftime('%m/%d/%Y') # convert to American date format
    except:
        date = ''
    res.append(date)
    times = re.findall('[0-9]{1,2}\:[0-9]{1,2}\:[0-9]{1,2}|[0-9]{1,2}\.[0-9]{1,2}\:[0-9]{1,2}|[0-9]{1,2}\.[0-9]{1,2}\.[0-9]{1,2}|[0-9]{1,2}\:[0-9]{1,2}\.[0-9]{1,2}', text)
    times = [time[1:] if time[0]=='0' else time for time in times]
    try:
        entry_time = times[0].replace('.',':')
    except IndexError:
        entry_time =''
    try:
        exit_time = times[1].replace('.',':')
    except IndexError:
        exit_time = ''
    res.append(entry_time)
    res.append(exit_time)
    weights = re.findall('\d+\ \d+ Kg|\d+ Kg|\d+\ \d+ kg|\d+ kg', text)
    weights = [int(w.replace('.','').replace('Kg','').replace('kg','').replace(' ','')) for w in weights]
    try:
        potongan = int(re.findall('\n[0-9]{3}\n', text)[0].replace('\n', ''))
    except:
        potongan = ''
    while len(weights) < 4:
        weights.append('')
    weights = weights[1:3] + [weights[0]] + [potongan] + [weights[3]]
    res += weights
    return res

In [20]:
def parsing_bss(text):
    res = []
    dates = re.findall('[0-9]{1,2}\-[0-9]{1,2}\-[0-9]{4}', text)
    try:
        date = dates[0]
        date = datetime.strptime(date, '%d-%m-%Y').strftime('%m/%d/%Y') # convert to American date format
    except IndexError:
        date = ''
    res.append(date)
    times = re.findall('[0-9]{1,2}\:[0-9]{1,2}\:[0-9]{1,2}', text)
    times = [time[1:] if time[0]=='0' else time for time in times]
    try:
        entry_time = times[0].replace('.',':')
    except IndexError:
        entry_time =''
    try:
        exit_time = times[1].replace('.',':')
    except IndexError:
        exit_time = ''
    res.append(entry_time)
    res.append(exit_time)
    weights = re.findall('\d+\,[0-9]{3}|\d+\.[0-9]{3}', text)
    try:
        potongan = re.findall('\n[0-9]{3} G|\n[0-9]{3} Ko|\n[0-9]{3} ng|\n[0-9]{3} K', text)[-1].replace(' G', '').replace(' Ko', '').replace(' ng', '').replace(' K', '').replace('\n', '')
    except:
        try:
            potongan = re.findall('\n[0-9]{3}\n', text)[0].replace('\n', '')
        except:
            potongan = ''
    while len(weights) < 4:
        weights.append('')
    weights = [w.replace('.','').replace(' kg','').replace(',','') for w in weights]
    weights = weights[:3] + [potongan] + [weights[3]]
    weights = [int(w) if w!='' else np.nan for w in weights]
    weights.sort()
    if np.nan not in weights:
        weights = [weights[i] for i in [4,1,3,0,2]]
    else:
        weights = [weights[3]] + [weights[4]] + [weights[2]] + [weights[0]] + [weights[1]]
    while len(weights) < 5:
        weights.append('')
    res += weights
    return res

In [21]:
def parsing_srjnad(text):
    res = []
    dates = re.findall('[0-9]{1,2}\/[0-9a-zA-Z]{1,2}\/[0-9]{4}|[0-9]{1,2}\/[0-9]{1,2}[1]{1}[0-9]{4}', text)
    try:
        date = dates[0]
        if date[3] == 'D':
            date = date[:3] + '0' + date[4:]
        if date[-5] == '1':
            date = date[:-5] + '/' + date[-4:]
        try:
            date = datetime.strptime(date, '%d/%m/%Y').strftime('%m/%d/%Y') # convert to American date format
        except:
            date = ''
    except IndexError:
        date = ''
    res.append(date)
    res.append('')
    res.append('')
    weights = re.findall('\d+\,[0-9]{3} Kg|[0-9]{3} Kg', text)
    weights = [int(w.replace(',','').replace(' Kg','')) for w in weights]
    if len(weights) == 6:
        weights = weights[1:]
    if len(weights) < 5 and len(weights) > 2:
        weights = weights[:3] + [''] + [weights[-1]]
    while len(weights) < 5:
        weights.append('')
    res += weights
    return res

In [22]:
function_mapping = {'arvena': parsing_arvena,
                    'bss': parsing_bss,
                    'nhr': parsing_nhr,
                    'skip': parsing_skip,
                    'srjnad': parsing_srjnad}

### Read json files, extract information and create tables for each mill

In [23]:
def extract_json(mill, ids=None):
    '''extracts IDs, users and text contents for this mill
    if ids are specified, only extracts these indices, otherwise extracts all available content'''
    directory = '../Images/weight_receipt/'
    all_images = [file for file in os.listdir(directory+mill) if file.endswith('.jpg')]
    if not ids:
        ids = [image.split('_')[0] for image in all_images]
    users = []
    for ID in ids:
        for image in all_images:
            if image.split('_')[0] == ID:
                users.append(image.split('_')[1])
    with open(directory+'/'+mill+'_text_contents.json') as f:
        text_contents = json.load(f)
    return ids, users, text_contents

#### Save table containing mill, ID, user, date, entry time, exit time and five weights

In [24]:
mills = ['arvena', 'bss', 'nhr','skip', 'srjnad']
for mill in mills:
    ids, users, all_texts = extract_json(mill)
    col_names = ['Mill', 'ID', 'User (middle man)', 'Date', 'Entry time', 'Exit time',
                 'First weight', 'Second weight', 'Net 1', 'Potongan', 'Net 2']
    df_mill_algo = pd.DataFrame([], columns = col_names)
    for ID, user in zip(ids, users):
        try:
            text = all_texts[ID]
            res = [mill, ID, user]
            res += function_mapping[mill](text)
            df_mill_algo.loc[len(df_mill_algo)] = res
        except:
            print(ID, 'not found in json file', mill)
    df_mill_algo.to_csv('../Images/weight_receipt/Algo '+mill+'.csv')

2539 not found in json file nhr
3671 not found in json file nhr
1907 not found in json file skip
3061 not found in json file skip
3223 not found in json file skip
3319 not found in json file srjnad


### Save results of algorithm to csv for all mills
Aggregate tables for all mills into one table  
Only use the entry time as the reference time  
Remove seconds from times  
Include date and time of receipt upload

In [25]:
df_upload = pd.read_csv('../Images/weight_receipt/date_time_upload.csv', index_col=0)
mills = ['arvena', 'bss', 'nhr','skip', 'srjnad']
col_names = ['Mill', 'ID', 'User (middle man)', 'Date', 'Entry time', 'Exit time',
                 'First weight', 'Second weight', 'Net 1', 'Potongan', 'Net 2']
df_all_mills = pd.DataFrame([], columns = col_names)
for mill in mills:
    df = pd.read_csv('../Images/weight_receipt/Algo '+mill+'.csv', index_col=0)
    df['Mill'] = mill
    df_all_mills = pd.concat([df_all_mills, df], sort=False)
df_all_mills['ID'] = df_all_mills['ID'].apply(lambda x: str(x))
df_upload['ID'] = df_upload['ID'].apply(lambda x: str(x))
df_all_mills = df_all_mills.merge(df_upload, on='ID')
def rm_sec(x):
    try:
        return x[:-3]
    except:
        return x
df_all_mills['Entry time'] = df_all_mills['Entry time'].apply(lambda x: rm_sec(x))
df_all_mills['Exit time'] = df_all_mills['Exit time'].apply(lambda x: rm_sec(x))
df_all_mills = df_all_mills.drop(columns=['Exit time']).rename(columns={'Entry time':'Time'})
df_all_mills['ID'] = df_all_mills['ID'].apply(lambda x: int(x))
df_all_mills = df_all_mills.sort_values(by=['ID'])
df_all_mills.to_csv('../Images/weight_receipt/Algo all mills.csv', index=False)
df_all_mills

,Mill,ID,User (middle man),Date,Time,First weight,Second weight,Net 1,Potongan,Net 2,Date created,Time created
570,nhr,9,dwisuyanto,NaN,NaN,10130.0,5605.0,NaN,NaN,NaN,12/05/2018,15:37
1842,srjnad,14,madyani,12/04/2018,NaN,11820.0,3910.0,7910.0,280.0,7630.0,12/05/2018,19:56
1861,srjnad,16,madyani,12/05/2018,NaN,9150.0,3890.0,5260.0,NaN,5050.0,12/05/2018,20:00
240,nhr,20,dwisuyanto,12/06/2018,15:09,11430.0,4320.0,7110.0,356.0,6754.0,12/06/2018,15:35
995,skip,23,aguswibowo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/07/2018,15:27
...,...,...,...,...,...,...,...,...,...,...,...,...
161,arvena,5719,muhammadfahrirambe,09/07/2019,10:48,6070.0,2900.0,8970.0,520.0,0.0,09/07/2019,12:56
2470,srjnad,5721,sudiwarnopandiangan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09/07/2019,15:23
1614,skip,5723,sumaji,09/07/2019,19:09,10290.0,3660.0,6630.0,332.0,6298.0,09/07/2019,15:58
1615,skip,5728,madyani,09/07/2019,16:29,12200.0,3910.0,8290.0,415.0,7875.0,09/07/2019,19:09
